### 比赛简介

心电图是检查心脏疾病常见手段之一，本次比赛使用PTB心电图数据集，输入电压：±16 mV，输入电阻：100Ω（DC），分辨率：16位，带宽：0-1 kHz。数据集包含心跳正常、心跳异常两种类型，选手需判断分类，正常=0，不正常=1。

数据集包含16个输入，其中14个心跳数据，1个呼吸数据，1个电压数据

频率：每秒1000次扫描

分辨率：16位

带宽：0-1 kHz

### 评审标准

我们将会对比选手提交的csv文件，确认正确识别样本数据


True：模型分类正确数量
Total ：测试集样本总数量

### 数据格式查看

In [1]:
import os
import pandas as pd

train_df = pd.read_csv("./ptbdb_train.csv", header=None)
train_df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,178,179,180,181,182,183,184,185,186,187
0,1.000000,0.888060,0.448561,0.166045,0.125533,0.118603,0.083955,0.063966,0.055437,0.056770,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.991337,0.594059,0.291460,0.000000,0.045173,0.123762,0.174505,0.195545,0.176980,0.225866,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.950226,0.975113,0.967195,0.993213,0.997738,1.000000,0.919683,0.809955,0.813348,0.745475,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1.000000,0.557010,0.290867,0.178592,0.165212,0.121873,0.111111,0.095695,0.087551,0.084933,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.000000,0.474798,0.191612,0.040015,0.096960,0.137745,0.143901,0.127741,0.123894,0.128511,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


一共188列，前187列为输入，最后一列为标签。0表示正常，1表示不正常。

In [2]:
train_df[187].value_counts()

1.0    3504
0.0    3496
Name: 187, dtype: int64

可以看出正常和不正常的标签分布平衡。

### 代码

**1.导入所需模块**

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow import keras
import tensorflow.keras.backend as K
from tensorflow.keras import Input
from tensorflow.keras.layers import Layer, Conv1D, Conv2D, MaxPool1D, MaxPool2D, Dense, Concatenate, RepeatVector, Reshape
from tensorflow.keras.layers import GlobalAveragePooling1D, GlobalAveragePooling2D, Multiply, Flatten, Dropout
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import pearsonr

import os

**2.加载训练集和待预测的数据**

In [4]:
# 加载数据
train_file = np.array(pd.read_csv("./ptbdb_train.csv", header=None))
test_file = np.array(pd.read_csv("./ptbdb_test.csv", header=None))

# 准备训练与测试集
train_x = train_file[:, :-1].reshape(-1, 1, 187, 1)
train_y = train_file[:, -1]

test_x = test_file.reshape(-1, 1, 187, 1)

del train_file, test_file

**3.定义一个inception_block块**

In [5]:
def inception_block(inputs, filters):
    """
    Inception Block模块
    :param inputs:输入 Tensor
    :param filters:滤波器个数
    :return 输出 Tensor
    """ 
    # 分支1
    conv_1 = Conv2D(filters=filters, kernel_size=[1, 1], strides=[1, 1], padding="same", activation="relu")(inputs)
    

    # 分支2
    conv_2 = Conv2D(filters=filters, kernel_size=[1, 3], strides=[1, 1], padding="same", activation="relu")(inputs)

    # 分支3
    conv_3 = Conv2D(filters=filters, kernel_size=[1, 3], strides=[1, 1], padding="same", activation="relu")(inputs)
    conv_3 = Conv2D(filters=filters, kernel_size=[1, 3], strides=[1, 1], padding="same", activation="relu")(conv_3)

    # 合并
    outputs = Concatenate(axis=3)([conv_1, conv_2, conv_3])
    outputs = Conv2D(filters=filters, kernel_size=[1, 1], strides=[1, 1], padding="same", activation="relu")(outputs)

    return outputs

**4.定义一个se_block模块**

In [6]:
def se_block(inputs, k):
    """
    SE Block模块
    :param inputs:输入 Tensor
    :param k:重标定重要性的参数
    :return 输出 Tensor
    """
    # 输入尺寸
    input_shape = K.int_shape(inputs)

    # 全局平均池化
    outputs = GlobalAveragePooling2D()(inputs)

    # 计算每个通道的重要性
    outputs = Dense(units=int(input_shape[-1] / k), activation="relu")(outputs)
    outputs = Dense(units=input_shape[-1], activation="sigmoid")(outputs)
    
    # 重新标定每个通道
    outputs = RepeatVector(input_shape[1] * input_shape[2])(outputs)
    outputs = Reshape([input_shape[1], input_shape[2], input_shape[3]])(outputs)
    outputs = Multiply()([inputs, outputs])
    
    return outputs

**5.定义一个特征提取层**

In [7]:
def ims_layer(inputs, filters, pool_size):
    """
    特征提取层
    :param inputs:输入 Tensor
    :param filters滤波器个数
    :return 输出 Tensor
    """
    inception = inception_block(inputs, filters)
    pool = MaxPool2D(pool_size=pool_size, strides=pool_size, padding="same")(inception)
    se = se_block(pool, 4)
    
    return se  

**6.定义全连接层**

In [8]:
def fc_layer(inputs, units):
    """
    全连接层
    :param inputs:输入 Tensor
    :return 输出 Tensor
    """
    outputs = Dense(units=units, activation="relu")(inputs)
    outputs = Dropout(0.5)(outputs)
    
    return outputs

**7.构建模型**

In [9]:
def base_model():
    """
    构建模型
    :return 模型句柄
    """
    # 原始输入数据
    x_input = Input(train_x.shape[1:])
    
    # 对原始信号提取特征
    # ims_1
    ims_1 = ims_layer(x_input, 64, [1, 2]) 
    
    # ims_2
    ims_2 = ims_layer(ims_1, 128, [1, 4]) 
    
    # Flatten
    flatten = Flatten()(ims_2)
    
    # fc
    # fc_1
    fc_1 = fc_layer(flatten, 512)
    # fc_2
    fc_2 = fc_layer(fc_1, 128)
                        
    # y_output
    y_output = Dense(units=1, activation="sigmoid")(fc_2)
    
    # 建立模型
    model = Model(inputs=x_input, outputs=y_output)
    # 编译模型
    model.compile(Adam(1e-3), loss='binary_crossentropy', metrics=['binary_accuracy'])
    
    return model

In [10]:
base_model().summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1, 187, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 1, 187, 64)   256         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 1, 187, 64)   128         input_1[0][0]                    
________________________________________________________________________________

**8.K折交叉验证模型模型**

In [10]:
k_fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)

model_list = []

accuracy_list = []

n_fold = 1

for train_index, val_index in k_fold.split(train_x, train_y):
    print("n_flod:{}.".format(n_fold))
    # 训练数据
    X_train = train_x[train_index]
    Y_train = train_y[train_index]
    # 验证数据
    X_val = train_x[val_index]
    Y_val = train_y[val_index]
    
    # 建立模型
    m = base_model()
    
    if os.path.exists("./model/cnn1_{}.h5".format(n_fold)):
        m.load_weights("./model/cnn1_{}.h5".format(n_fold))
    else:
        # 早停函数
        early_stop = EarlyStopping(monitor="val_loss", min_delta=0, patience=50, mode="auto", restore_best_weights=True)
        # 训练
        m.fit(X_train, Y_train, batch_size=128, validation_data=(X_val, Y_val), epochs=5000, callbacks=[early_stop])
        # 保存
        m.save_weights("./model/cnn1_{}.h5".format(n_fold))
    
    n_fold += 1
    
    # 评估
    _, accuracy = m.evaluate(X_val, Y_val, verbose=0)
    print("当前模型正确率为:{}.".format(accuracy))
    
    # 存储模型
    model_list.append(m)
    
    # 存储单个模型的正确率
    accuracy_list.append(accuracy)

print("平均正确率为:{}.".format(np.mean(accuracy_list)))

n_flod:1.
Train on 5599 samples, validate on 1401 samples
Epoch 1/5000
5599/5599 [==============================] - 3s 449us/sample - loss: 0.5693 - binary_accuracy: 0.7276 - val_loss: 0.4766 - val_binary_accuracy: 0.7830
Epoch 2/5000
5599/5599 [==============================] - 2s 285us/sample - loss: 0.4295 - binary_accuracy: 0.8184 - val_loss: 0.4108 - val_binary_accuracy: 0.8287
Epoch 3/5000
5599/5599 [==============================] - 2s 282us/sample - loss: 0.3692 - binary_accuracy: 0.8557 - val_loss: 0.3470 - val_binary_accuracy: 0.8651
Epoch 4/5000
5599/5599 [==============================] - 2s 285us/sample - loss: 0.2986 - binary_accuracy: 0.8855 - val_loss: 0.2903 - val_binary_accuracy: 0.8844
Epoch 5/5000
5599/5599 [==============================] - 2s 286us/sample - loss: 0.2431 - binary_accuracy: 0.9080 - val_loss: 0.2592 - val_binary_accuracy: 0.8908
Epoch 6/5000
5599/5599 [==============================] - 2s 284us/sample - loss: 0.2014 - binary_accuracy: 0.9243 - val_l

5599/5599 [==============================] - 2s 290us/sample - loss: 0.0029 - binary_accuracy: 0.9987 - val_loss: 0.0665 - val_binary_accuracy: 0.9886
Epoch 100/5000
5599/5599 [==============================] - 2s 291us/sample - loss: 0.0044 - binary_accuracy: 0.9986 - val_loss: 0.0579 - val_binary_accuracy: 0.9893
Epoch 101/5000
5599/5599 [==============================] - 2s 290us/sample - loss: 0.0044 - binary_accuracy: 0.9989 - val_loss: 0.0492 - val_binary_accuracy: 0.9921
Epoch 102/5000
5599/5599 [==============================] - 2s 289us/sample - loss: 0.0037 - binary_accuracy: 0.9986 - val_loss: 0.0345 - val_binary_accuracy: 0.9936
Epoch 103/5000
5599/5599 [==============================] - 2s 284us/sample - loss: 8.5662e-04 - binary_accuracy: 0.9996 - val_loss: 0.0551 - val_binary_accuracy: 0.9929
Epoch 104/5000
5599/5599 [==============================] - 2s 291us/sample - loss: 6.3358e-04 - binary_accuracy: 0.9998 - val_loss: 0.0462 - val_binary_accuracy: 0.9907
Epoch 105/5

Epoch 7/5000
5600/5600 [==============================] - 2s 288us/sample - loss: 0.1533 - binary_accuracy: 0.9402 - val_loss: 0.1622 - val_binary_accuracy: 0.9407
Epoch 8/5000
5600/5600 [==============================] - 2s 291us/sample - loss: 0.1234 - binary_accuracy: 0.9546 - val_loss: 0.1419 - val_binary_accuracy: 0.9507
Epoch 9/5000
5600/5600 [==============================] - 2s 286us/sample - loss: 0.0969 - binary_accuracy: 0.9664 - val_loss: 0.1545 - val_binary_accuracy: 0.9543
Epoch 10/5000
5600/5600 [==============================] - 2s 291us/sample - loss: 0.0899 - binary_accuracy: 0.9673 - val_loss: 0.1464 - val_binary_accuracy: 0.9543
Epoch 11/5000
5600/5600 [==============================] - 2s 291us/sample - loss: 0.0710 - binary_accuracy: 0.9748 - val_loss: 0.1531 - val_binary_accuracy: 0.9593
Epoch 12/5000
5600/5600 [==============================] - 2s 289us/sample - loss: 0.0765 - binary_accuracy: 0.9696 - val_loss: 0.1409 - val_binary_accuracy: 0.9607
Epoch 13/5000

Epoch 57/5000
5600/5600 [==============================] - 2s 290us/sample - loss: 0.0083 - binary_accuracy: 0.9970 - val_loss: 0.1330 - val_binary_accuracy: 0.9836
Epoch 58/5000
5600/5600 [==============================] - 2s 286us/sample - loss: 0.0063 - binary_accuracy: 0.9977 - val_loss: 0.1644 - val_binary_accuracy: 0.9800
Epoch 59/5000
5600/5600 [==============================] - 2s 291us/sample - loss: 0.0074 - binary_accuracy: 0.9977 - val_loss: 0.1362 - val_binary_accuracy: 0.9829
Epoch 60/5000
5600/5600 [==============================] - 2s 286us/sample - loss: 0.0035 - binary_accuracy: 0.9991 - val_loss: 0.1491 - val_binary_accuracy: 0.9850
Epoch 61/5000
5600/5600 [==============================] - 2s 286us/sample - loss: 0.0028 - binary_accuracy: 0.9993 - val_loss: 0.1789 - val_binary_accuracy: 0.9857
Epoch 62/5000
5600/5600 [==============================] - 2s 293us/sample - loss: 0.0041 - binary_accuracy: 0.9993 - val_loss: 0.1630 - val_binary_accuracy: 0.9807
Epoch 63/5

5600/5600 [==============================] - 2s 289us/sample - loss: 0.1265 - binary_accuracy: 0.9532 - val_loss: 0.1306 - val_binary_accuracy: 0.9564
Epoch 9/5000
5600/5600 [==============================] - 2s 292us/sample - loss: 0.1148 - binary_accuracy: 0.9586 - val_loss: 0.1038 - val_binary_accuracy: 0.9686
Epoch 10/5000
5600/5600 [==============================] - 2s 290us/sample - loss: 0.1008 - binary_accuracy: 0.9655 - val_loss: 0.0933 - val_binary_accuracy: 0.9714
Epoch 11/5000
5600/5600 [==============================] - 2s 292us/sample - loss: 0.0828 - binary_accuracy: 0.9693 - val_loss: 0.0850 - val_binary_accuracy: 0.9700
Epoch 12/5000
5600/5600 [==============================] - 2s 296us/sample - loss: 0.0656 - binary_accuracy: 0.9764 - val_loss: 0.0791 - val_binary_accuracy: 0.9764
Epoch 13/5000
5600/5600 [==============================] - 2s 292us/sample - loss: 0.0671 - binary_accuracy: 0.9746 - val_loss: 0.0860 - val_binary_accuracy: 0.9771
Epoch 14/5000
5600/5600 [

Epoch 58/5000
5600/5600 [==============================] - 2s 293us/sample - loss: 0.0085 - binary_accuracy: 0.9971 - val_loss: 0.1367 - val_binary_accuracy: 0.9757
Epoch 59/5000
5600/5600 [==============================] - 2s 291us/sample - loss: 0.0099 - binary_accuracy: 0.9964 - val_loss: 0.1006 - val_binary_accuracy: 0.9829
Epoch 60/5000
5600/5600 [==============================] - 2s 291us/sample - loss: 0.0063 - binary_accuracy: 0.9984 - val_loss: 0.0958 - val_binary_accuracy: 0.9879
Epoch 61/5000
5600/5600 [==============================] - 2s 289us/sample - loss: 0.0019 - binary_accuracy: 0.9996 - val_loss: 0.0892 - val_binary_accuracy: 0.9879
Epoch 62/5000
5600/5600 [==============================] - 2s 293us/sample - loss: 0.0034 - binary_accuracy: 0.9989 - val_loss: 0.0764 - val_binary_accuracy: 0.9900
Epoch 63/5000
5600/5600 [==============================] - 2s 293us/sample - loss: 0.0036 - binary_accuracy: 0.9986 - val_loss: 0.0890 - val_binary_accuracy: 0.9879
Epoch 64/5

5601/5601 [==============================] - 2s 293us/sample - loss: 0.0616 - binary_accuracy: 0.9802 - val_loss: 0.1067 - val_binary_accuracy: 0.9657
Epoch 14/5000
5601/5601 [==============================] - 2s 293us/sample - loss: 0.0460 - binary_accuracy: 0.9850 - val_loss: 0.0720 - val_binary_accuracy: 0.9764
Epoch 15/5000
5601/5601 [==============================] - 2s 292us/sample - loss: 0.0363 - binary_accuracy: 0.9859 - val_loss: 0.0660 - val_binary_accuracy: 0.9771
Epoch 16/5000
5601/5601 [==============================] - 2s 292us/sample - loss: 0.0351 - binary_accuracy: 0.9866 - val_loss: 0.1228 - val_binary_accuracy: 0.9607
Epoch 17/5000
5601/5601 [==============================] - 2s 297us/sample - loss: 0.0425 - binary_accuracy: 0.9850 - val_loss: 0.0594 - val_binary_accuracy: 0.9821
Epoch 18/5000
5601/5601 [==============================] - 2s 302us/sample - loss: 0.0312 - binary_accuracy: 0.9904 - val_loss: 0.0529 - val_binary_accuracy: 0.9843
Epoch 19/5000
5601/5601 

Epoch 63/5000
5601/5601 [==============================] - 2s 293us/sample - loss: 0.0088 - binary_accuracy: 0.9964 - val_loss: 0.0534 - val_binary_accuracy: 0.9900
Epoch 64/5000
5601/5601 [==============================] - 2s 294us/sample - loss: 0.0074 - binary_accuracy: 0.9979 - val_loss: 0.0439 - val_binary_accuracy: 0.9900
Epoch 65/5000
5601/5601 [==============================] - 2s 293us/sample - loss: 0.0064 - binary_accuracy: 0.9979 - val_loss: 0.0607 - val_binary_accuracy: 0.9893
Epoch 66/5000
5601/5601 [==============================] - 2s 296us/sample - loss: 0.0030 - binary_accuracy: 0.9991 - val_loss: 0.0554 - val_binary_accuracy: 0.9921
Epoch 67/5000
5601/5601 [==============================] - 2s 294us/sample - loss: 0.0021 - binary_accuracy: 0.9993 - val_loss: 0.0622 - val_binary_accuracy: 0.9914
Epoch 68/5000
5601/5601 [==============================] - 2s 292us/sample - loss: 0.0048 - binary_accuracy: 0.9982 - val_loss: 0.0566 - val_binary_accuracy: 0.9921
Epoch 69/5

5601/5601 [==============================] - 2s 291us/sample - loss: 0.0046 - binary_accuracy: 0.9989 - val_loss: 0.0496 - val_binary_accuracy: 0.9921
Epoch 162/5000
5601/5601 [==============================] - 2s 294us/sample - loss: 0.0027 - binary_accuracy: 0.9991 - val_loss: 0.0457 - val_binary_accuracy: 0.9943
Epoch 163/5000
5601/5601 [==============================] - 2s 293us/sample - loss: 2.8914e-04 - binary_accuracy: 1.0000 - val_loss: 0.0442 - val_binary_accuracy: 0.9929
Epoch 164/5000
5601/5601 [==============================] - 2s 291us/sample - loss: 3.2571e-04 - binary_accuracy: 0.9998 - val_loss: 0.0466 - val_binary_accuracy: 0.9936
Epoch 165/5000
5601/5601 [==============================] - 2s 291us/sample - loss: 1.6197e-04 - binary_accuracy: 1.0000 - val_loss: 0.0457 - val_binary_accuracy: 0.9936
Epoch 166/5000
5601/5601 [==============================] - 2s 292us/sample - loss: 1.2329e-04 - binary_accuracy: 1.0000 - val_loss: 0.0497 - val_binary_accuracy: 0.9950
Epo

**9.对带预测的数据进行预测**

In [11]:
# 当前模型的预测结果
res_list = []

for m in model_list:
    res_list.append(m.predict(test_x))
    
res = np.round(np.mean(res_list, axis=0), 0).astype("float")

**10.提交文件生成**

In [12]:
# 生成当前模型的提交文件
pd.DataFrame({"id": [i for i in range(0, res.shape[0])], "label": res.reshape(-1, )}).to_csv("./sub_cnn.csv", 
                                                                                             header=None, index=None)